In [2]:
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *
from lasagne.objectives import *
from lasagne.regularization import *
from lasagne.random import get_rng
from lasagne.updates import *
from lasagne.init import *
import numpy as np
import sys
sys.path.append("../../modules/")
import helper as hp

from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

import matplotlib.pyplot as plt
%matplotlib inline

import os
import cPickle as pickle

from theano.tensor import TensorType

from theano.ifelse import ifelse

from time import time

%load_ext rpy2.ipython

from scipy import stats

import deep_residual_learning_CIFAR10

//anaconda/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [5]:
srng = RandomStreams(get_rng().randint(1, 2147462579))

In [13]:
srng.binomial(n=1, p=0.5, size=(10,), dtype="float32").dimshuffle(0,'x','x','x').eval().shape

(10, 1, 1, 1)

In [14]:
srng.binomial(n=1, p=0.5, size=(10,), dtype="float32").dimshuffle(0,'x','x','x').eval() * np.ones( (10, 3, 2, 2) )

array([[[[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]]],


       [[[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]]],


       [[[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]]],


       [[[ 1.,  1.],
         [ 1.,  1.]],

        [[ 1.,  1.],
         [ 1.,  1.]],

        [[ 1.,  1.],
         [ 1.,  1.]]],


       [[[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]]],


       [[[ 1.,  1.],
         [ 1.,  1.]],

        [[ 1.,  1.],
         [ 1.,  1.]],

        [[ 1.,  1.],
         [ 1.,  1.]]],


       [[[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.]]],


       [[[ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.

Stuff related to resnet shortcut connections...

In [31]:
l_in = InputLayer( (None, 3, 28, 28) )
l_conv1 = Conv2DLayer(l_in, num_filters=8, filter_size=3, pad='same')
l_conv2 = Conv2DLayer(l_conv1, num_filters=8, filter_size=3,pad='same')
print l_conv2.output_shape

(None, 8, 28, 28)


In [34]:
Conv2DLayer(l_in, num_filters=8, stride=2, filter_size=1, pad=0).output_shape

(None, 8, 14, 14)

https://github.com/Lasagne/Lasagne/issues/531

In [69]:
def residual_block(layer, num_filters, filter_size=3, stride=1, num_layers=2):
    print layer.output_shape
    conv = layer
    if (num_filters != layer.output_shape[1]) or (stride != 1):
        layer = Conv2DLayer(layer, num_filters, filter_size=1, stride=stride, pad=0, nonlinearity=None, b=None)
    for _ in range(num_layers):
        # first conv has to respect the stride, so that we can add the id connection
        conv = Conv2DLayer(conv, num_filters, filter_size, stride=stride, pad='same')
        print conv.output_shape
        stride = 1
    print "[conv, layer] = %s, %s" % (str(conv.output_shape), str(layer.output_shape))
    return ElemwiseSumLayer([conv, layer])

In [70]:
l_out = residual_block(l_in, num_filters=8, filter_size=3, stride=2)

(None, 3, 28, 28)
(None, 8, 14, 14)
(None, 8, 14, 14)
[conv, layer] = (None, 8, 14, 14), (None, 8, 14, 14)


In [55]:
X = T.tensor4('X')

In [59]:
out_fn = theano.function([X], get_output(l_out, X))

In [61]:
out_fn( np.ones((1,3,28,28) ) )

ValueError: Input dimension mis-match. (input[1].shape[2] = 28, input[3].shape[2] = 14)
Apply node that caused the error: Elemwise{Composite{((i0 * (Abs((i1 + i2)) + i1 + i2)) + i3)}}[(0, 1)](TensorConstant{(1, 1, 1, 1) of 0.5}, CorrMM{half, (1, 1)}.0, InplaceDimShuffle{x,0,x,x}.0, CorrMM{(0, 0), (2, 2)}.0)
Toposort index: 9
Inputs types: [TensorType(float64, (True, True, True, True)), TensorType(float64, 4D), TensorType(float64, (True, False, True, True)), TensorType(float64, 4D)]
Inputs shapes: [(1, 1, 1, 1), (1, 8, 28, 28), (1, 8, 1, 1), (1, 8, 14, 14)]
Inputs strides: [(8, 8, 8, 8), (50176, 6272, 224, 8), (64, 8, 8, 8), (12544, 1568, 112, 8)]
Inputs values: [array([[[[ 0.5]]]]), 'not shown', 'not shown', 'not shown']
Outputs clients: [['output']]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can 

----

In [149]:
import math

In [209]:
l_in = InputLayer( (None, 3, 28, 28), name="input" )
l_conv1 = Conv2DLayer(l_in, num_filters=8, filter_size=3, name="conv1")
l_conv2 = Conv2DLayer(l_conv1, num_filters=8, filter_size=3, name="conv2")
for layer in get_all_layers(l_conv2):
    print layer.name, layer.output_shape

input (None, 3, 28, 28)
conv1 (None, 8, 26, 26)
conv2 (None, 8, 24, 24)


In [210]:
id1 = Pool2DLayer(l_conv1, pool_size=(2,2), mode="average_inc_pad")
print id1.output_shape

(None, 8, 13, 13)


In [211]:
# if the pool shape is not divisible by 2, let's pad asymmetrically
# e.g. if (13,13) then make it (14,14)
if id1.output_shape[2] % 2 == 1:
    l_pad = pad(id1, width=((0,1), (0,1)))
# now, pad it so that it's good
nd1 = (l_conv2.output_shape[2]-l_pad.output_shape[2])/2
l_pad = pad(l_pad, width=(nd1, nd1))
    
l_pad.output_shape

(None, 8, 24, 24)

In [199]:
out_pad = theano.function([X], get_output(l_pad, X))

In [201]:
out_pad( np.ones((10,3,28,28)) ).shape

(10, 8, 24, 24)

-----

In [370]:
l_in = InputLayer( (None, 3, 28, 28), name="input" )
l_conv1 = Conv2DLayer(l_in, num_filters=8, filter_size=3, name="conv1")
l_conv2 = Conv2DLayer(l_conv1, num_filters=8, filter_size=3, name="conv2")
for layer in get_all_layers(l_conv2):
    print layer.name, layer.output_shape

input (None, 3, 28, 28)
conv1 (None, 8, 26, 26)
conv2 (None, 8, 24, 24)


In [619]:
class StochasticDepthLayer(Layer):
    def __init__(self, incoming, in_var, nonlinearity=rectify, p=0.5,
                 **kwargs):
        super(StochasticDepthLayer, self).__init__(incoming, **kwargs)
        self.nonlinearity = (identity if nonlinearity is None
                             else nonlinearity)
        self._srng = RandomStreams(get_rng().randint(1, 2147462579))
        self.p = p
        
        self.layer_before_incoming = incoming.input_layer
        #for prev_layer in get_all_layers(incoming)[::-1]:
        #    if isinstance(prev_layer, Conv2DLayer) or isinstance(prev_layer, Pool2DLayer) \
        #    or isinstance(prev_layer, InputLayer):
        #        self.layer_before_incoming = prev_layer
        #        break
        
        self.incoming = incoming
        self.in_var = in_var

    def get_output_for(self, input, deterministic=False, **kwargs):
        if deterministic or self.p == 0.0:
            return input
        else:
            """
            layer_before_incoming: (26, 26)
            incoming: (24, 24)
            If we apply avg pooling to layer_before_incoming, we get:
            (13,13)
            So let's add a condition to pad asymmetrically so that it is:
            (14,14)
            Then, let's pool up to (24, 24)
            """
            layer_before_incoming = self.layer_before_incoming
            incoming = self.incoming
            print "layer i have right now: %s" % str(incoming.name),
            print "layer before incoming: %s" % str(layer_before_incoming.name)
            l_pool = Pool2DLayer(layer_before_incoming, pool_size=(2,2), mode="average_inc_pad")
            print "l_pool: %s" % str(l_pool.output_shape),
            if (l_pool.output_shape[2] % 2 == 1 and incoming.output_shape[2] % 2 == 0) or \
                (l_pool.output_shape[2] % 2 == 0 and incoming.output_shape[2] % 2 == 1):
                l_pad = pad( l_pool, width=((0,1),(0,1)) )
                print "l_pad: %s" % str(l_pad.output_shape),
            else:
                l_pad = l_pool
                print "l_pad: %s" % str(l_pad.output_shape),
            nd1 = (self.incoming.output_shape[2]-l_pad.output_shape[2])/2
            print "nd1: %i" % nd1,
            if nd1 > 0:
                l_pad = pad(l_pad, width=(nd1,nd1))
            # what if the layer_before_incoming num feature maps is
            # less than the incoming_layer num feature maps?
            if layer_before_incoming.output_shape[1] < incoming.output_shape[1]:
                diff_in_fms = incoming.output_shape[1]-layer_before_incoming.output_shape[1]
                if diff_in_fms % 2 == 0: 
                    width_tp = ((diff_in_fms/2, diff_in_fms/2),)
                else:
                    width_tp = (((diff_in_fms/2)+1, diff_in_fms/2),)
                l_pad = pad(
                    l_pad, 
                    batch_ndim=1, 
                    width=width_tp
                ) 
            
            print "l_pad: %s" % str(l_pad.output_shape)

            l_pad_out = get_output(l_pad, self.in_var)

            # ok, now do the binomial mask
            mask = self._srng.binomial(n=1, p=(self.p), size=(input.shape[0],),
                dtype=input.dtype)
            mask = mask.dimshuffle(0,'x','x','x')
            
            return self.nonlinearity( (mask*input) + l_pad_out )
            #return self.nonlinearity(input+l_pad_out)

In [372]:
l_test = StochasticDepthLayer(l_conv2, nonlinearity=rectify, in_var=X)
l_test.output_shape

(None, 8, 24, 24)

In [373]:
fn_test = theano.function([X], get_output(l_test, X))

yes
<function rectify at 0x107833140>


In [376]:
fn_test( np.ones((1, 3, 28, 28)) )

array([[[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ..., 
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],

        [[ 0.02005802,  0.02005802,  0.02005802, ...,  0.02005802,
           0.02005802,  0.02005802],
         [ 0.02005802,  0.02005802,  0.02005802, ...,  0.02005802,
           0.02005802,  0.02005802],
         [ 0.02005802,  0.02005802,  0.02005802, ...,  0.02005802,
           0.02005802,  0.02005802],
         ..., 
         [ 0.02005802,  0.020058

In [620]:
l_in = InputLayer( (None, 3, 28, 28), name="input" )
l_conv1 = StochasticDepthLayer(
    Conv2DLayer(l_in, num_filters=8, filter_size=3, name="conv1", nonlinearity=None),
    nonlinearity=rectify, in_var=l_in.input_var, name="sd_conv1"
)
l_mp1 = StochasticDepthLayer(
    MaxPool2DLayer(l_conv1, pool_size=(2,2), name="pool1"),
    nonlinearity=identity, in_var=l_in.input_var, name="sd_mp1"
)
l_conv2 = StochasticDepthLayer(
    Conv2DLayer(l_mp1, num_filters=16, filter_size=3, name="conv2", nonlinearity=None),
    nonlinearity=rectify, in_var=l_in.input_var, name="sd_conv2"
)
l_mp2 = StochasticDepthLayer(
    MaxPool2DLayer(l_conv2, pool_size=(2,2), name="pool2"),
    nonlinearity=identity, in_var=l_in.input_var, name="sd_mp2"
)
l_conv3 = StochasticDepthLayer(
    Conv2DLayer(l_mp2, num_filters=32, filter_size=3, name="conv3", nonlinearity=None),
    nonlinearity=rectify, in_var=l_in.input_var, name="sd_conv3"
)
l_out = DenseLayer(l_conv3, num_units=10, nonlinearity=softmax, name="final")

for layer in get_all_layers(l_out):
    print layer.name, layer.output_shape

input (None, 3, 28, 28)
conv1 (None, 8, 26, 26)
sd_conv1 (None, 8, 26, 26)
pool1 (None, 8, 13, 13)
sd_mp1 (None, 8, 13, 13)
conv2 (None, 16, 11, 11)
sd_conv2 (None, 16, 11, 11)
pool2 (None, 16, 5, 5)
sd_mp2 (None, 16, 5, 5)
conv3 (None, 32, 3, 3)
sd_conv3 (None, 32, 3, 3)
final (None, 10)


In [616]:
fn_test = theano.function([X], get_output(l_out, X))

layer i have right now: conv1 layer before incoming: input
l_pool: (None, 3, 14, 14) l_pad: (None, 3, 14, 14) nd1: 6 l_pad: (None, 8, 26, 26)
layer i have right now: pool1 layer before incoming: sd_conv1
l_pool: (None, 8, 13, 13) l_pad: (None, 8, 13, 13) nd1: 0 l_pad: (None, 8, 13, 13)
layer i have right now: conv1 layer before incoming: input
l_pool: (None, 3, 14, 14) l_pad: (None, 3, 14, 14) nd1: 6 l_pad: (None, 8, 26, 26)
layer i have right now: conv2 layer before incoming: sd_mp1
l_pool: (None, 8, 6, 6) l_pad: (None, 8, 7, 7) nd1: 2 l_pad: (None, 16, 11, 11)
layer i have right now: conv1 layer before incoming: input
l_pool: (None, 3, 14, 14) l_pad: (None, 3, 14, 14) nd1: 6 l_pad: (None, 8, 26, 26)
layer i have right now: pool1 layer before incoming: sd_conv1
l_pool: (None, 8, 13, 13) l_pad: (None, 8, 13, 13) nd1: 0 l_pad: (None, 8, 13, 13)
layer i have right now: conv1 layer before incoming: input
l_pool: (None, 3, 14, 14) l_pad: (None, 3, 14, 14) nd1: 6 l_pad: (None, 8, 26, 26)
la

MissingInputError: ("An input of the graph, used to compute Pool{ds=(2, 2), ignore_border=True, st=(2, 2), padding=(0, 0), mode='average_inc_pad'}(input.input), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.", input.input)

In [617]:
fn_test( np.ones((10, 3, 28, 28) ) ).shape

(10, 10)

In [608]:
np.sum(np.isnan( np.ones((10, 3, 28, 28) ) ))

0

In [594]:
sys.stderr.write("loading mnist...\n")
train_data, valid_data, _ = hp.load_mnist("../../data/mnist.pkl.gz")

loading mnist...


In [595]:
X_train, y_train = train_data

In [596]:
X_train_subset = X_train[0:200]
y_train_subset = y_train[0:200]

In [ ]:
X = l_in.input_var
y = T.ivector('y')


net_out = get_output(l_out, X)
loss = categorical_crossentropy(net_out, y).mean()
params = get_all_params(l_out, trainable=True)
grads = T.grad(loss, params)
updates = nesterov_momentum(grads, params, learning_rate=0.01, momentum=0.9)
train_fn = theano.function(inputs=[X, y], outputs=loss, updates=updates)


layer i have right now: conv1 layer before incoming: input
l_pool: (None, 3, 14, 14) l_pad: (None, 3, 14, 14) nd1: 6 l_pad: (None, 8, 26, 26)
layer i have right now: pool1 layer before incoming: sd_conv1
l_pool: (None, 8, 13, 13) l_pad: (None, 8, 13, 13) nd1: 0 l_pad: (None, 8, 13, 13)
layer i have right now: conv1 layer before incoming: input
l_pool: (None, 3, 14, 14) l_pad: (None, 3, 14, 14) nd1: 6 l_pad: (None, 8, 26, 26)
layer i have right now: conv2 layer before incoming: sd_mp1
l_pool: (None, 8, 6, 6) l_pad: (None, 8, 7, 7) nd1: 2 l_pad: (None, 16, 11, 11)
layer i have right now: conv1 layer before incoming: input
l_pool: (None, 3, 14, 14) l_pad: (None, 3, 14, 14) nd1: 6 l_pad: (None, 8, 26, 26)
layer i have right now: pool1 layer before incoming: sd_conv1
l_pool: (None, 8, 13, 13) l_pad: (None, 8, 13, 13) nd1: 0 l_pad: (None, 8, 13, 13)
layer i have right now: conv1 layer before incoming: input
l_pool: (None, 3, 14, 14) l_pad: (None, 3, 14, 14) nd1: 6 l_pad: (None, 8, 26, 26)
la